# Lobito Corridor Spatial Analysis
----

Benny Istanto, bistanto@worldbank.org, GOST/DEC Data Group

Objective. Produce fast, reproducible **access & opportunity** analytics using only aligned rasters: 
1. Isochrone coverage KPIs (population, cropland, electrification), 
2. A transparent **priority surface** for smallholder aggregation/feeder upgrades, 
3. A **flood-aware road risk** screening, and 
4. **Site cards** for the project locations.
              
**Expected outputs.** AOI-prefixed GeoTIFFs in `outputs/rasters/`, CSV tables in `outputs/tables/`, and quick PNG maps in `outputs/figs/`.

## Run once after (re)starting the kernel


<div class="alert alert-block alert-success" align="left">
  <b>Why this cell?</b><br/>
  It initializes the workspace for the current AOI—sets <code>PROJECT_ROOT</code>/<code>AOI</code>, adds <code>src/</code> to <code>sys.path</code>, clears the import cache, and loads <code>config.py</code> so <code>PATHS</code>/<code>PARAMS</code> are available to all steps.
  <br/><br/>
  <b>When to run it</b>
  <ul>
    <li>After a kernel restart (once).</li>
    <li>After changing <code>AOI_VALUE</code> or editing <code>config.py</code> (paths/parameters/templates).</li>
    <li>After moving the project folder.</li>
  </ul>
  <b>Editing step code?</b> Re-import the step and <code>importlib.reload(module)</code> before calling <code>main()</code>.
</div>


In [1]:
# Bootstrap imports for package-style repo
import os, sys, importlib
from pathlib import Path

# >>>> EDIT THESE TWO ONLY <<<<
PROJECT_ROOT = Path("/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis")
AOI_VALUE = "moxicoleste"  # e.g., "benguela", "huambo", "bei", "moxico", "moxicoleste"

# Environment for the codebase
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
os.environ["AOI"] = AOI_VALUE

src_path = PROJECT_ROOT / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Make sure Python sees any file edits we made
importlib.invalidate_caches()

from config import PATHS, PARAMS, AOI, get_logger
print("AOI:", AOI)
PATHS.OUT, PATHS.OUT_R, PATHS.OUT_T, PATHS.OUT_F


AOI: moxicoleste


(PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs'),
 PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/rasters'),
 PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables'),
 PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/figs'))

## Step 00 — Data alignment & vector→raster

Align everything to a single 1-km template so every pixel stacks perfectly. This step converts key vectors (cropland, electrification, settlements) to rasters, harmonizes units/CRS, and prepares a 1-km flood screening layer alongside the native 30 m flood depth for engineering checks.

**Objective.** Produce AOI-aligned base rasters and vector-to-raster layers for consistent downstream math.

**Inputs.**
* `PARAMS.TARGET_GRID` (template), raw rasters (POP/NTL/VEG/drought), flood depth (30 m), and vectors (cropland, electrification, settlements).
  
**Outputs.**
* `{AOI}_pop_1km.tif`, `{AOI}_ntl_1km.tif`, `{AOI}_veg_1km.tif`, `{AOI}_drought_1km.tif`,
* `{AOI}_cropland_1km.tif`, `{AOI}_electric_1km.tif`, `{AOI}_settlement_1km.tif`,
* `{AOI}_flood_maxdepth_30m.tif` + `{AOI}_flood_maxdepth_1km.tif`,
* `{AOI}_rwi_meta_1km.tif` (if available in your data bundle).
  
**Assumptions & design.**
* Everything is reprojected/matched to the template; vector rasterization uses `all_touched=True`.
* Nodata is respected; logging warns on missing optional layers.

**Knobs.** 
* In `config.py`: source paths, `PARAMS.TARGET_GRID`.


In [2]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_00_align_and_rasterize"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:40:53 | INFO | Loaded target grid | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083 (units of CRS)
06:40:53 | INFO | Aligning rasters to target grid...
06:40:54 | INFO | Wrote moxicoleste_pop_1km.tif
06:40:54 | INFO | Wrote moxicoleste_ntl_1km.tif
06:40:54 | INFO | Wrote moxicoleste_veg_1km.tif
06:40:54 | INFO | Wrote moxicoleste_drought_1km.tif
06:40:54 | INFO | Aligning Meta Relative Wealth Index (RWI) to 1-km grid...
06:40:55 | INFO | Wrote moxicoleste_rwi_meta_1km.tif
06:40:55 | INFO | Rasterizing cropland presence (1=any in cell)...
06:40:58 | INFO | Wrote moxicoleste_cropland_presence_1km.tif
06:40:58 | INFO | Rasterizing cropland fraction (0..1 per 1-km cell, supersample=10)...
06:41:00 | INFO | Wrote moxicoleste_cropland_fraction_1km.tif | mean_fraction=0.001
06:41:01 | INFO | Rasterizing electricity masks using field 'FinalElecC' (1=grid, 99=unelectrified)...
06:41:01 | INFO | Wrote moxicoleste_elec_grid_1km.tif, moxicoleste_elec_unelectrified_1km.tif
06:41:02 | INFO | Ra

## Step 01 — Isochrones (accumulated travel time)

Turn road/friction inputs into a travel-time surface in minutes. This becomes the backbone for coverage masks (≤30/60/120 min) and access KPIs.

**Objective.** Compute the accumulated cost (minutes) raster.
                                                                                                               
**Inputs.**
* `PARAMS.TARGET_GRID`, friction/network inputs per `config.py`.
  
**Outputs.**
* `{AOI}_traveltime_min.tif` (accumulated cost surface).
  
**Assumptions.**
* Minutes as the canonical unit; obstacles/penalties embedded in friction.
  
**Knobs.**
* Friction parameters in `PARAMS` (if exposed), thresholds in `PARAMS.ISO_THRESH` (e.g., `(30, 60, 120)`).


In [3]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_01_isochrones"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:14 | INFO | Isochrones from ago_phy_moxicoleste_traveltime_market.tif | CRS=EPSG:4326 | size=342x408 | cell=0.0x0.0
06:41:14 | INFO | Building ≤30 min isochrone...
06:41:14 | INFO | Wrote moxicoleste_iso_le_30min_1km.tif | cells inside: 1250/86094 (1.5%)
06:41:14 | INFO | Building ≤60 min isochrone...
06:41:14 | INFO | Wrote moxicoleste_iso_le_60min_1km.tif | cells inside: 3014/86094 (3.5%)
06:41:14 | INFO | Building ≤120 min isochrone...
06:41:14 | INFO | Wrote moxicoleste_iso_le_120min_1km.tif | cells inside: 8529/86094 (9.9%)
06:41:14 | INFO | Building ≤240 min isochrone...
06:41:15 | INFO | Wrote moxicoleste_iso_le_240min_1km.tif | cells inside: 25239/86094 (29.3%)
06:41:15 | INFO | Step 01 complete.


## Step 02 — KPIs for isochrone coverage (population, cropland, electrification)

Quantify how much population, cropland, and electrification lie within each time threshold. These coverage KPIs are the simplest way to communicate “who’s connected.”

**Objective.** Summarize POP/CROP/ELECTRIC inside `tt ≤ threshold`.

**Inputs.**
* `{AOI}_traveltime_min.tif`, 1-km POP/CROPLAND/ELECTRIC rasters.
  
**Outputs.**
* `tables/{AOI}_iso_kpis.csv` (per threshold).
  
**Assumptions.**
* Coverage mask: `tt_minutes <= threshold`.
  
**Knobs.**
* `PARAMS.ISO_THRESH` list.


In [4]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_02_kpis_population_cropland_electric"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:15 | INFO | Electrification grid value counts (non-NaN): {}
06:41:15 | INFO | KPI base rasters loaded | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:15 | INFO | Electrification mapping: FinalElecCode2020 -> elec_grid_1km == 1 (grid); 99/0/NaN = non-grid.
06:41:15 | INFO | Denominators | pop_total=372,044 | cropland_total_km2=51.86 | electrified_cells_total=0 | cell_area_km2≈1.000
06:41:15 | INFO | ≤ 30 min | cells=1,250 (area=1,250.00 km²) | pop=246,630 (66.3%) | crop=16.57 km² (32.0%) | elec_cells=0 (nan%)
06:41:15 | INFO | ≤ 60 min | cells=3,014 (area=3,014.00 km²) | pop=271,239 (72.9%) | crop=21.19 km² (40.9%) | elec_cells=0 (nan%)
06:41:15 | INFO | ≤120 min | cells=8,529 (area=8,529.00 km²) | pop=306,828 (82.5%) | crop=25.13 km² (48.5%) | elec_cells=0 (nan%)
06:41:15 | INFO | ≤240 min | cells=25,239 (area=25,239.00 km²) | pop=345,154 (92.8%) | crop=25.79 km² (49.7%) | elec_cells=0 (nan%)
06:41:15 | INFO | Saved KPI table → /mnt/c/Users/benny/OneDrive/Documents/Gith

## Step 03 — Priority surface (baseline)

Combine normalized indicators: better access (lower minutes), higher population & vegetation, stronger nighttime lights, and lower drought frequency. Save the score raster and a top-decile mask to highlight quick-win areas.

**Objective.** (Baseline) Build a transparent, minimal priority surface from core overlays.

**Inputs.**
* 1-km standardized layers (POP/CROP/ELECTRIC etc.).
  
**Outputs.**
* `rasters/{AOI}_priority_score_0_1.tif`, `rasters/{AOI}_priority_top10_mask.tif`.
  
**Assumptions.**
* Simpler normalization and weights vs Step 07.
  
**Recommendation.**
* Keep Step 03 as **baseline** only; prefer Step 07 for production.


In [5]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_03_priority_surface"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:15 | INFO | Priority surface inputs loaded | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:15 | INFO | Weights | ACC=0.35 POP=0.25 VEG=0.20 NTL=0.10 DRT=0.10
06:41:16 | INFO | Wrote moxicoleste_priority_score_v1_0_1.tif
06:41:16 | INFO | Wrote moxicoleste_priority_top10_mask_v1.tif | P90=0.578 | top10 cells: 55/544 (10.1%)
06:41:16 | INFO | Step 03 complete.


## Step 04 — Flood bottlenecks from road raster

Flag where roads intersect flood depths beyond thresholds. Use the 30 m layer for engineering plausibility, and the 1 km “max depth” for corridor-scale screening.

**Objective.** Identify likely flood bottlenecks impacting road access.

**Inputs.**
* 30 m flood depth, road raster or vector.
  
**Outputs.**
* `rasters/{AOI}_flood_bottlenecks_1km.tif` (or CSV summary, depending on implementation).
  
**Assumptions.**
* Engineering check: depth thresholds configurable.
  
**Knobs.**
* Flood depth threshold(s) in `PARAMS` (if exposed).


In [6]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_04_flood_bottlenecks_from_road_raster"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:16 | WARNING | Reprojected to match target grid: flood1k, flood_frac, prio10
06:41:16 | INFO | Flood screening inputs loaded | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:16 | INFO | Params | FLOOD_DEPTH_RISK=0.3 m | ROAD_CLASSES_KEEP=None | ROADS_ALL_TOUCHED=False
06:41:16 | INFO | Rasterizing roads: no class filter (use all OSM fclass values).
06:41:17 | INFO | Wrote moxicoleste_roads_main_1km.tif | road_cells=7,227
06:41:17 | INFO | Wrote moxicoleste_roads_flood_risk_cells_1km.tif | risk_cells=4,391
06:41:17 | INFO | Wrote moxicoleste_roads_flood_risk_near_priority_1km.tif | risk_near_priority_cells=51
06:41:17 | INFO | Saved summary → /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables/moxicoleste_roads_flood_risk_summary.csv
06:41:17 | INFO | Step 04 complete.


## Step 05 — Site audit points

Build “site cards” by sampling rasters around project points. Quicklook stats and thumbnails help triage which locations deserve deeper design work.

**Objective.** Generate site-level diagnostic metrics and snapshots.
    
**Inputs.**
* Project points (CSV/GeoPackage as per `config.py`).
  
**Outputs.**
* `tables/{AOI}_site_audit_points.csv` (+ optional PNGs in `outputs/figs/`).
  
**Assumptions.**
* Points in EPSG:4326; rasters sampled with nearest or bilinear.
  
**Knobs.**
* Sampling radius / list of rasters to sample (if exposed).


In [7]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_05_site_audit_points"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
importlib.reload(m)

m.main()


06:41:17 | WARNING | Step 05: site audit will be EMPTY.
  Reason: site layer not found: /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/data/vectors/ago_poi_moxicoleste_projectloc_dm_p.shp
  This is expected if the AOI has no project locations.
  Downstream steps should handle an empty site audit table.
06:41:17 | INFO | Saved EMPTY site audit → /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables/moxicoleste_site_audit_points.csv | rows=0 | skipped_outside_grid=0
06:41:17 | INFO | Step 05 complete (no sites).


## Step 06 — Municipality (Admin2) ingest & correlations

Ingest Admin2 (RAPP) themes, standardize units, rasterize selected variables, and test correlations versus rural poverty. Produces a tidy “profiles” table and 1-km Admin2 surfaces usable in later steps.

**Objective.** Normalize Admin2 indicators and rasterize key variables.
    
**Inputs.**
* `data/vectors/ago_*_rapp_2020_a.shp` (themes in `THEME_VARS`), `PARAMS.TARGET_GRID`.
  
**Outputs.**
* `tables/{AOI}_municipality_indicators.csv` (wide),
* `tables/{AOI}_corr_with_rural_poverty.csv`,
* `tables/{AOI}_municipality_profiles.csv`,
* `rasters/{AOI}_muni_{theme}_{var}_1km.tif`.
  
**Assumptions & design.**
* Percentages normalized to 0–1 (`RAPP_PCT_IS_0_100=True`); travel time hours → minutes; `all_touched=True`.
* Missing variables skipped with warnings.
  
**Knobs.**
* `THEME_VARS` mapping, list of variables to rasterize.


In [8]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_06_muni_ingest"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:17 | INFO | Target grid loaded | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:17 | INFO | Step 06 will skip themes: ('climevents',)
06:41:17 | INFO | Theme=waterresources: read=0.10s, rename=0.00s, normalize=0.00s, rows=4, cols=10
06:41:17 | INFO | Theme=communications: read=0.10s, rename=0.00s, normalize=0.00s, rows=4, cols=11
06:41:17 | INFO | Theme=infra: read=0.10s, rename=0.00s, normalize=0.01s, rows=4, cols=15
06:41:17 | INFO | Theme=foodinsecurity: read=0.09s, rename=0.00s, normalize=0.01s, rows=4, cols=14
06:41:17 | INFO | Theme=outflow: read=0.11s, rename=0.00s, normalize=0.01s, rows=4, cols=12
06:41:18 | INFO | Theme=poverty: read=0.10s, rename=0.00s, normalize=0.00s, rows=4, cols=8
06:41:18 | INFO | Theme=productions: read=0.10s, rename=0.00s, normalize=0.01s, rows=4, cols=15
06:41:18 | INFO | Traveltime theme: converted 'avg_hours_to_market_financial' (hours) → 'avg_minutes_to_market_financial' (minutes).
06:41:18 | INFO | Theme=traveltime: read=0.10s, rena

/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/src/step_06_muni_ingest.py:240: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = pearsonr(x[m].values.astype(float), y[m].values.astype(float))


06:41:19 | INFO | Wrote moxicoleste_muni_communications_telephone_1km.tif | min=0.0000 max=0.2450
06:41:19 | INFO | Wrote moxicoleste_muni_communications_internet_1km.tif | min=0.0000 max=0.0430
06:41:19 | INFO | Wrote moxicoleste_muni_communications_newspaper_1km.tif | min=0.0000 max=0.0070
06:41:19 | INFO | Wrote moxicoleste_muni_communications_radio_1km.tif | min=0.0000 max=0.2650
06:41:19 | INFO | Wrote moxicoleste_muni_communications_television_1km.tif | min=0.0000 max=0.1020
06:41:19 | INFO | Wrote moxicoleste_muni_communications_none_1km.tif | min=0.0000 max=0.6730
06:41:20 | INFO | Wrote moxicoleste_muni_foodinsecurity_went_without_food_1km.tif | min=0.0000 max=0.9840
06:41:20 | INFO | Wrote moxicoleste_muni_foodinsecurity_unable_eat_healthy_1km.tif | min=0.0000 max=0.2570
06:41:20 | INFO | Wrote moxicoleste_muni_foodinsecurity_few_types_of_food_1km.tif | min=0.0000 max=0.8250
06:41:20 | INFO | Wrote moxicoleste_muni_foodinsecurity_skipped_meal_1km.tif | min=0.0000 max=0.9010
0

## Step 07 — Priority surface (tunable, authoritative)

Create the **production** priority surface with coherent normalization, optional equity/poverty overlays, focal smoothing, tiny-cluster removal, and a Top-X% mask.

**Objective.** Build the authoritative priority map and mask with flexible weights.

**Inputs.**
* 1-km base layers + optional Admin2 rasters (auto-discovered), including `rwi_meta_1km` if present.
  
**Outputs.**
* `rasters/{AOI}_priority_score_0_1.tif`, `rasters/{AOI}_priority_top10_mask.tif`.
  
**Assumptions & design.**
* Unified normalization, masks, focal smoothing, small-cluster removal, Top-X selection.
  
**Knobs.**
* Weights and Top-X% in `PARAMS`/module (e.g., `PARAMS.TOP_X_PCT`).
  
**Note.**
* Downstream steps (11, 12) expect **these** canonical filenames from `config.py`.


In [9]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_07_priority_tunable"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config, utils_geo
# importlib.reload(config)
# importlib.reload(utils_geo)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above./
# importlib.reload(m)

m.main()


06:41:24 | INFO | Target grid | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:26 | INFO | Reprojected raster to match grid: POP
06:41:26 | INFO | Reprojected raster to match grid: VEG
06:41:26 | INFO | Reprojected raster to match grid: NTL
06:41:26 | INFO | Reprojected raster to match grid: DRT
06:41:26 | INFO | Reprojected raster to match grid: CROP
06:41:26 | INFO | Reprojected raster to match grid: RURAL
06:41:26 | INFO | Reprojected overlay to match grid: moxicoleste_rwi_meta_1km.tif
06:41:26 | INFO | Reprojected overlay to match grid: moxicoleste_muni_poverty_poverty_rural_1km.tif
06:41:26 | INFO | Reprojected overlay to match grid: moxicoleste_muni_foodinsecurity_food_insec_scale_1km.tif
06:41:27 | INFO | Reprojected overlay to match grid: moxicoleste_muni_traveltime_avg_minutes_to_market_financial_1km.tif
06:41:27 | INFO | Priority weight blend → ACC:0.29, POP:0.21, DRT:0.08, POV:0.12, FOOD:0.08, MTT:0.08, RWI:0.12
06:41:27 | INFO | Wrote moxicoleste_priority_score_0_1

## Step 08 — Project-level KPIs

Aggregate access/exposure/equity metrics over project buffers or polygons. Useful for comparing alternatives under the same data stack.

**Objective.** Compute KPI summaries for each project footprint/buffer.
    
**Inputs.**
* Project geometries + aligned rasters (POP/CROP/ELECTRIC/PRIORITY/etc.).
  
**Outputs.**
* `tables/{AOI}_project_kpis.csv`.
  
**Assumptions.**
* Buffers in meters; stats computed with consistent nodata handling.
  
**Knobs.**
* Buffer distance; KPI list (in step module).


In [10]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_08_project_kpis"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config, utils_geo
# importlib.reload(config)
# importlib.reload(utils_geo)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:27 | INFO | Template grid | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:27 | INFO | Reprojected pop to match grid
06:41:27 | INFO | Reprojected cropf to match grid
06:41:27 | INFO | Reprojected grid to match grid
06:41:27 | INFO | Reprojected rural to match grid
06:41:27 | INFO | Reprojected prio to match grid
06:41:27 | INFO | Reprojected risk to match grid
06:41:27 | INFO | Reprojected pov to match grid
06:41:27 | INFO | Denominators | pop_total=372,044 | cropland_total_km2=43.65 | electrified_cells_total=0 | cell_area_km2=0.842
06:41:27 | WARNING | No project points found (SITES/GOV/WB/OTH).
06:41:27 | INFO | No projects found. Wrote empty table → moxicoleste_project_kpis.csv


## Step 09 — Municipality (Admin2) targeting

Rank Admin2 units using access (≤60/120 min), poverty/equity signals, and priority shares. Produces an actionable shortlist for piloting.

**Objective.** Score and rank Admin2s for investment targeting.

**Inputs.**
* Admin2 rasters and tabular indicators from Step 06 + travel-time surface.
  
**Outputs.**
* `tables/{AOI}_muni_targeting.csv` (+ optional choropleth figs).
  
**Assumptions.**
* Uses `%≤60/≤120m` coverage, priority mask share, etc.
  
**Knobs.**
* Thresholds in `PARAMS.ISO_THRESH`; ranking weights in the module.


In [11]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_09_muni_targeting"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:28 | INFO | Template grid | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:28 | INFO | Using admin2 geometry from ago_pop_moxicoleste_adm2_poverty_rapp_2020_a.shp
06:41:28 | INFO | Saved municipality targeting table → moxicoleste_priority_muni_rank.csv | rows=4
06:41:28 | INFO | Step 09 complete.


## Step 10 — Priority scenarios

Stress-test the priority model with alternative weight sets (e.g., stronger equity vs. market access) and compare outcomes.

**Objective.** Generate and record scenario variants of the priority surface.
    
**Inputs.**
* Same base overlays as Step 07, different weight/toggle sets.
  
**Outputs.**
* `rasters/{AOI}_priority_score_scnX.tif`, masks, and a `tables/{AOI}_priority_scenarios.csv`.
  
**Assumptions.**
* Scenarios defined in code or `PARAMS`.
  
**Knobs.**
* Scenario configs (weights/toggles) in the module or `PARAMS`.


In [12]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_10_priority_scenarios"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:28 | INFO | Target grid | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:29 | INFO | Using on-disk baseline: moxicoleste_priority_top10_mask.tif
06:41:29 | INFO | Scenario [baseline] — ACC+POP+DRT, rural-only, min crop 5%, 3x3 smooth, Top10%
06:41:29 | INFO | Scenario [no_drought] — ACC+POP only; rural-only; min crop 5%; Top10%
06:41:29 | INFO | Scenario [veg_signal] — ACC+POP+VEG; rural-only; min crop 10%; Top10%
06:41:29 | INFO | Scenario [top_800km2] — ACC+POP+DRT, rural-only; select fixed 800 km²; mild smooth
06:41:29 | INFO | Wrote scenario summary → moxicoleste_priority_scenarios_summary.csv | scenarios=4 | masks_saved=4
06:41:29 | INFO | Wrote scenarios sidecar → moxicoleste_priority_scenarios.meta.json
06:41:29 | INFO | Step 10 complete.


## Step 11 — Priority clusters

Convert the Top-X% priority mask into connected clusters and compute cluster KPIs (population, cropland, RWI, and share within ≤30/60/120 min). Small, noisy blobs are removed using cell and km² thresholds.

**Objective.** Extract cluster polygons/labels and summarize them for shortlisting.
    
**Inputs.**
* `PRIORITY_TOP10_TIF` from `config.py`, `traveltime_min` + base rasters, optional `rwi_meta_1km`.
  
**Outputs.**
* `rasters/{AOI}_priority_clusters_1km.tif` (via `PRIORITY_CLUSTERS_TIF`),
* `tables/{AOI}_priority_clusters.csv`.
  
**Assumptions & design.**
* Prune tiny clusters by min cells and **km²** (now configurable via `PARAMS.MIN_CLUSTER_KM2`).
* Added KPIs: `share_le_30m/60m/120m` (uses `PARAMS.ISO_THRESH`).
  
**Knobs.**
* `PARAMS.MIN_CLUSTER_CELLS`, `PARAMS.MIN_CLUSTER_KM2`, `PARAMS.ISO_THRESH`.


In [13]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_11_priority_clusters"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:29 | INFO | Template grid | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:30 | INFO | Connected components found: 0
06:41:30 | INFO | Saved clusters → moxicoleste_priority_clusters.csv | clusters=0
06:41:30 | INFO | Step 11 complete.


## Step 12 — Travel-time catchments

Build catchment masks for each threshold and compute POP/CROP/RWI KPIs plus mean travel minutes—a quick validation of how “tight” the catchments are.

**Objective.** Derive per-threshold catchment KPIs and diagnostics.
    
**Inputs.**
* `{AOI}_traveltime_min.tif`, 1-km POP/CROPLAND, optional `rwi_meta_1km`.
  
**Outputs.**
* `tables/{AOI}_catchments_kpis.csv` (via `CATCHMENTS_KPI_CSV`),
* per-threshold masks in `rasters/` (if the step writes them).
  
**Assumptions & design.**
* Coverage = `tt <= thr`; includes `mean_travel_min` diagnostic.
  
**Knobs.**
* `PARAMS.ISO_THRESH`.


In [14]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_12_traveltime_catchments"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:30 | INFO | Template grid | CRS=EPSG:4326 | size=342x408 | cell=0.0083x0.0083
06:41:30 | ERROR | Project points not found: /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/data/vectors/ago_poi_moxicoleste_projectloc_dm_p.shp


## Step 13 — Synergies overlay

Intersect priority with constraints/opportunities (e.g., flood risk, grid proximity, conservation). Useful to surface “no-regret” or “high-risk” zones.

**Objective.** Produce an overlay table (and optional rasters) capturing key synergies/tradeoffs.

**Inputs.**
* Priority rasters, flood/constraint rasters, grid/market proximity if available.
  
**Outputs.**
* `tables/{AOI}_synergies_overlay.csv`, optionally composited rasters.
  
**Assumptions.**
* Binary or categorical overlay logic documented in the step.
  
**Knobs.**
* Overlay weights/toggles in the module.


In [15]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_13_synergies_overlay"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
importlib.reload(m)

m.main()


06:41:30 | WARNING | SITES layer missing for AOI=moxicoleste. Site-level synergies will be empty. (SITES not found: /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/data/vectors/ago_poi_moxicoleste_projectloc_dm_p.shp)
06:41:30 | WARNING | Step 13: writing EMPTY site_synergies table.
  Reason: SITES shapefile not found for this AOI.
  Path  : /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables/moxicoleste_site_synergies.csv
06:41:30 | INFO | gov layer not found: ago_poi_moxicoleste_projects_gov_p.shp
06:41:30 | INFO | wb layer not found: ago_poi_moxicoleste_projects_wb_p.shp
06:41:30 | INFO | oth layer not found: ago_poi_moxicoleste_projects_others_p.shp
06:41:30 | INFO | No sites available → skipping site-level synergies.
06:41:30 | INFO | Cluster CSV found but missing centroid columns (or empty); will derive centroids from raster.
06:41:30 | WARNING | Step 13: writing EMPTY cluster_synergies table.
  Reason: No clusters pres

## Step 14 — Origin-Destination (Admin2 gravity)

Sketch flows between Admin2 zones using a gravity model for corridor storytelling. Supports equity tilting (RWI), selectable impedance (exponential or power), distance cutoffs, and (optionally) doubly-constrained balancing.

**Objective.** Generate OD flows, zone attributes, and sampled agents for quick viz.
                                                                      
**Inputs.**
* `{AOI}_pop_1km.tif`, optional `rwi_meta_1km.tif`, Admin2 shapefile (`muni_path_for`).
  
**Outputs.**
* `tables/{AOI}_od_gravity.csv` (long: oi,dj,flow,dist_km),
* `tables/{AOI}_od_zone_attrs.csv` (zone masses + centroids),
* `tables/{AOI}_od_agents.csv` (sampled OD pairs for quick viz).
  
**Assumptions & design.**
* Geodesic centroid–centroid distances; **optional** RWI mass tilt;
* Impedance `f(D)` selectable (`exp` or `pow`); optional **doubly-constrained** via IPF; distance cutoff.
  
**Knobs (in `PARAMS`).**
* `OD_F`: `"exp"` or `"pow"`; `OD_LAMBDA` (exp), `OD_BETA` (pow),
* `OD_MAX_DIST_KM`, `OD_TRIPS_TOTAL`, `OD_USE_DOUBLY_CONSTRAINED`,
* `USE_RWI_IN_MASS`, `RWI_WEIGHT`, `OD_N_AGENTS`.


In [16]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_14_lite_od"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:41:31 | INFO | RWI found: using equity tilt.
06:41:31 | INFO | OD zones aligned by ADM2CD_c order | N=4
06:41:31 | INFO | Flows: total=1,000,000 | mean_dist_km=132.7 | within_cutoff%=99.99999999999999
06:41:31 | INFO | Wrote moxicoleste_od_gravity.csv, moxicoleste_od_zone_attrs.csv
06:41:31 | INFO | Wrote moxicoleste_od_agents.csv (N=16)
06:41:31 | INFO | Step 14 complete.


## End of Calculation